# Introduction <a name="introduction"></a>
The notebook will **create a classification model using Scikit-learn, log facts using autologging,save model to Space using Watson Machine Learning**, **link model to model usecase**, **get and set environment types** for saved model asset.  
<br/>
For more information around model helper utilities see [here](https://ibm-aigov-facts-client.mybluemix.net/#model-helpers)

It requires following services in the platform:

- Watson Studio
- Watson Machine Learning
- Watson Knowledge catalog
- AI factsheet
- Facts client python sdk (>=1.0.42)

Sections: 

- [Setup](#setup)
    - [Cloud](#setupcloud)
    - [Cloud Pak for data](#setupcpd)
- [Initialize client](#init)
    - [Cloud](#setupcloudinit)
    - [Cloud Pak for data](#setupcpdinit)
- [Create model using scikit learn](#createmodel)
- [Save model in Watson machine learning space](#savemodel)
- [Get model object](#getmodel)
- [Add model to model usecase ](#add_mu)
- [Get model current environment](#getenv)
- [Set model environment ](#remove_mu)
- [Cleanup](#clean)




<a name="setupcloud"></a>

# Setup <a href="#setup"></a>
### Imports


In [ ]:
!pip install -U ibm-aigov-facts-client --quiet
!pip install -U ibm-watson-machine-learning --quiet

In [1]:
import warnings
import shutil
import time
from ibm_watson_machine_learning import APIClient
from ibm_aigov_facts_client import AIGovFactsClient
from IPython.core.display import display, Markdown

warnings.filterwarnings('ignore')
shutil.rmtree('./mlruns', ignore_errors=True)

- This sample will use IBM Cloud by default. If you prefer to use Cloud pak for data, set `use_cp4d=True`
- Flag `run_cleanup_at_end` offers option to delete created assets at the end of the notebook.The notebook will show URL to UI for model and model use case at certain cells. By dafault we set it to `run_cleanup_at_end=False` so you can access UI and see the changes. If you decide to cleanup assets at the end, set `run_cleanup_at_end=True` and remember cells showing links to UI will `NOT` work in that case.

In [2]:
use_cp4d=False
run_cleanup_at_end=False

- Model container type can be `SPACE` or `PROJECT`. However to use get/set environment utilities in this notebook, model asset need to be promoted to Space first.

In [3]:
EXPERIMENT_NAME="IrisClassification"
MODEL_NAME="IrisScikitModel"
CONTAINER_TYPE="space"

## IBM Cloud <a href="#setupcloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [4]:
if not use_cp4d:
    API_KEY="***"
    SPACE_ID="***"

[back to top](#introduction)
## Cloud Pak for Data <a href="#setupcpd"></a>
- Service url is the Cloud pak for data platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user `password` or platform `apikey` to authenticate
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [5]:
if use_cp4d:
    from ibm_aigov_facts_client import CloudPakforDataConfig

    creds=CloudPakforDataConfig(service_url="***",
        username="***",
        api_key="***")
    
    SPACE_ID="***"


[back to top](#introduction)
# Client Initialization

## IBM Cloud <a href="#setupcloudinit"></a>

- Container type would be either `space` or `project`. To use get/set environment utilities, model asset should be stored in Space.
- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client

In [6]:
if not use_cp4d:
    facts_client = AIGovFactsClient(api_key=API_KEY, experiment_name= EXPERIMENT_NAME, container_type=CONTAINER_TYPE,container_id=SPACE_ID)

2022/12/12 18:02:11 INFO : Experiment successfully created with ID 1 and name IrisClassification
2022/12/12 18:02:12 INFO : Autolog enabled Successfully


## Cloud Pak for Data <a href="#setupcpdinit"></a>

- Service url is the Cloud pak for data platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user password or platform apikey to authenticate
- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client

In [7]:
if use_cp4d:
    facts_client = AIGovFactsClient(experiment_name= EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=SPACE_ID,cloud_pak_for_data_configs=creds)

[back to top](#introduction)
## Create model <a href="#createmodel"></a>

In [8]:
import pandas as pd
from sklearn import svm, datasets
import numpy as np

iris = datasets.load_iris()
data1 = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                    columns= ['sepal_length','sepal_width','petal_length','petal_width','target'])

from sklearn.model_selection import train_test_split
train, test = train_test_split(data1, test_size = 0.4, random_state = 42)

X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.target
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.target

svc = svm.SVC(kernel="rbf", C=2,probability=True)
model=svc.fit(X_train,y_train)

model.score(X_test, y_test)
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)


2022/12/12 18:02:14 INFO : logging results to factsheet for run_id ed6c56247dab41fda8ce45997e29e0e3
2022/12/12 18:02:16 INFO : Successfully logged results to Factsheet service for run_id ed6c56247dab41fda8ce45997e29e0e3 under asset_id: 9bc27910-3cd7-4799-bbbd-8ec6ee6d268b and space_id : 2f3d2c21-9b7e-4d8f-af5d-c65276c71318


[back to top](#introduction)
# Store model to Space using Watson Machine Learning <a href="#savemodel"></a>

In [9]:
if use_cp4d:
       WML_CREDENTIALS = {
                   "url": creds.url,
                   "username": creds.username,
                   "apikey" : creds.api_key,
                   "instance_id": "wml_local",
                   "version" : "4.5"
                  }
else:
       WML_CREDENTIALS = {
                     "url": "https://us-south.ml.cloud.ibm.com",
                     "apikey": API_KEY
              }

In [10]:
wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.253'

In [11]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [12]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
    wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
    wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.0",
    wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
    wml_client._models.ConfigurationMetaNames.LABEL_FIELD:"target",
}

facts_client.export_facts.prepare_model_meta(wml_client=wml_client,meta_props=model_props)

print("Storing model .....")

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=data1.drop(["target"], axis=1), training_target=data1.target)
model_id = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_id))

Software Specification ID: 12b83a17-24d8-5082-900f-0ab31fbfd3cb
Storing model .....
Done
Model ID: 72d97d15-1e9c-4246-95ac-fbfdb519168c


## Get model <a href="#getmodel"></a>
- It initates model object using Watson machine learning saved model details and returns a model object that allows you to use all helper methods [here](https://ibm-aigov-facts-client.mybluemix.net/#model-helpers)
- `verbose=True` gives you additional informations about model asset
- You can also use model_id to initate model object `facts_client.assets.get_model(model_id=<wml model id>)`

In [13]:
wml_model=facts_client.assets.get_model(wml_stored_model_details=published_model_details)
wml_model.get_info(verbose=True)

2022/12/12 18:02:23 INFO : Current model information: {'asset_id': '72d97d15-1e9c-4246-95ac-fbfdb519168c', 'container_type': 'space', 'container_id': '2f3d2c21-9b7e-4d8f-af5d-c65276c71318', 'facts_type': 'modelfacts_user'}


{'name': 'IrisScikitModel',
 'asset_type': 'wml_model',
 'url': 'https://dataplatform.cloud.ibm.com/ml-runtime/models/72d97d15-1e9c-4246-95ac-fbfdb519168c?space_id=2f3d2c21-9b7e-4d8f-af5d-c65276c71318&context=cpdaas',
 'asset_id': '72d97d15-1e9c-4246-95ac-fbfdb519168c',
 'container_type': 'space',
 'container_id': '2f3d2c21-9b7e-4d8f-af5d-c65276c71318',
 'facts_type': 'modelfacts_user'}

In [14]:
model_ui_url = wml_model.get_info(verbose=True)["url"]
display(Markdown("[Click here to see the created wml model details in UI](" + model_ui_url + ")"))

[Click here to see the created wml model details in UI](https://dataplatform.cloud.ibm.com/ml-runtime/models/72d97d15-1e9c-4246-95ac-fbfdb519168c?space_id=2f3d2c21-9b7e-4d8f-af5d-c65276c71318&context=cpdaas)

[back to top](#introduction)
## Add model to new model usecase <a href="#add_mu"></a>

- Model usecase helps to track model asset lifecycle across different environments like development,pre-production and production.
- You can link a model to existing model usecase by providing `model_usecase_id` or create a new usecase and link to that by providing `model_usecase_name` and `model_usecase_desc`.
- You can link to existing model usecase by invoking `external_model.add_tracking_model_usecase(model_usecase_id="<model usecase id>")`
- If `model_usecase_catalog_id` is not given, it uses tha default available platform asset catalog id.
- When you open the model usecase in model inventory, `model_usecase_id` refers to value after `asset/{ usecase id}` in browser URL.
- You can also invoke `get_tracking_model_usecase()` to return currently tracked model usecase and get model usecase details by invoking `get_tracking_model_usecase().get_info()`
- If using Cloud pak for data, make sure Openpages integration is disabled in your platform (create catalog permission needed). You can check it from UI `(Model inventory -> Manage)`.

In [15]:
wml_model.add_tracking_model_usecase(model_usecase_name="ModelUsecaseDraft",model_usecase_desc="Draft model usecase for testing")

2022/12/12 18:02:24 INFO : Initiate linking model to new model use case......
2022/12/12 18:02:38 INFO : Successfully finished linking Model 72d97d15-1e9c-4246-95ac-fbfdb519168c to model use case


{'model_entry_catalog_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
 'model_entry_id': 'aba647a6-f4ba-4133-bbe9-9e1c86f1263d',
 'model_entry_name': 'ModelUsecaseDraft',
 'model_entry_status': 'draft'}

In [16]:
model_usecase=wml_model.get_tracking_model_usecase()
model_usecase.get_info()

{'model_usecase_id': 'aba647a6-f4ba-4133-bbe9-9e1c86f1263d',
 'container_type': 'catalog',
 'catalog_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
 'facts_type': 'model_entry_user'}

[back to top](#introduction)
## Get model environment type <a href="#getenv"></a>
- When tracking is enabled, the recording of facts is triggered by actions on the model or related assets. Similarly, changes in the AI lifecycle control where the model display in a model usecase, to show the progression from development to operation. 
- For example, when a model is saved in a project, the model displays in the Develop pillar. When the model is promoted to a space, the entry is updated to show the model in the Test pillar, and so on.
- `get_environment_type()` shows you where your asset exists and the reason behind it.
- `TEST` environment is similar to `DEPLOY` in Cloud pak for data UI until `4.6.3` release. There is no difference in functionalities. From `4.6.3` onwards the environment will be called/showed as `TEST` in Cloud pak for data platform. 


Current expectations:

| Model location | Develop | Test (show as Deploy in CPD <=4.6.2) | Validate | Operate |
| --- | --- | --- |--- | --- |
| Project | ✓ |
| Space |  | ✓ |
| Space tagged with AIGovernance: Pre-production | | | ✓ |
| Space tagged with AIGovernance: Production | | | | ✓ |
| Watson OpenScale Pre-production | | |✓ |
| Watson OpenScale Production | | | | ✓ |

- For external models, space concept is not applicable, however the method works as is considering your external model is saved as model stub in `Platform asset catalog` and linked to model usecase. You can follow steps as follows:
    - Save mode stub using `save_external_model_asset()`. You can see example [here](https://ibm-aigov-facts-client.mybluemix.net/#externalmodelfactselements)
    - Add to model usecase invoking `add_tracking_model_usecase()`
    - Get current environment invoking `get_environment_type()`


In [17]:
wml_model.get_environment_type()

{'classification': 'TEST',
 'reason': 'The space type is development which is considered as TEST environment and asset shows under TEST stage'}

[back to top](#introduction)
## Set model environment type <a href="#remove_mu"></a>


Following notes describe how the interaction between the lifecycle components controls where a Watson Machine Learning displays in a model entry.

- When a model is promoted to a space, the model displays in the Test pillar. If the deployment for that model is evaluated in Watson OpenScale and tagged as pre-production, then the model displays in Validate rather than Test. If the model deployment is tagged as production instead of pre-production, then it moves to the Operate pillar.
- If you assign the space tag `AIGovernance: Pre-production` or `AIGovernance: Production`, the model displays in the Validate or Operate pillar, respectively. Note: This change is triggered by an action resulting in a change to the model metadata, such as updating the model name, description, or tags, if the tag is added to the space after the model is tracked.
- If a model is promoted to a space where the space has tag `AIGovernance: Pre-production`, the model displays in the Validate pillar. If the deployment for that model is evaluated in Watson OpenScale and tagged as pre-production, then the model displays in the Validate pillar. If the model deployment is tagged as production instead of pre-production, then it moves to the Operate pillar.
- If a model is promoted to a space where the space has the tag AIGovernance: Production, it displays in the Operate pillar. If the deployment for this model is evaluated in Watson OpenScale and tagged as production, then it will be still in Operate pillar. But if it is tagged as Pre-production it displays in the Validate pillar.
- If a model deployment is not evaluated in Watson OpenScale, it will have a Pending Evaluation tag. If the model deployment is evaluated, it will have an Evaluated tag. If the model deployment is approved after evaluation in Watson OpenScale, it will have an Approved tag.

Notes:
- If model is promoted to space and not monitored in Watson OpenScale, you can set model to specific environment pillar.
- For external models, bahavior is identical.

In [18]:
time.sleep(3)
wml_model.set_environment_type(from_container="test",to_container="validate")

# uncomment if running cell by cell to see changes in UI

# model_usecase_ui_url = model_usecase.get_info(verbose=True)["url"]
# display(Markdown("[Click here to see the model current environment in UI (Under Assets tab)](" + model_usecase_ui_url + ")"))

2022/12/12 18:02:46 INFO : Asset successfully moved from test to validate environment


In [19]:
time.sleep(3)
wml_model.set_environment_type(from_container="validate",to_container="operate")

# uncomment if running cell by cell to see changes in UI

# model_usecase_ui_url = model_usecase.get_info(verbose=True)["url"]
# display(Markdown("[Click here to see the model current environment in UI (Under Assets tab)](" + model_usecase_ui_url + ")"))

2022/12/12 18:02:52 INFO : Asset successfully moved from validate to operate environment


In [20]:
time.sleep(3)
wml_model.set_environment_type(from_container="operate",to_container="validate")

# uncomment if running cell by cell to see changes in UI

# model_usecase_ui_url = model_usecase.get_info(verbose=True)["url"]
# display(Markdown("[Click here to see the model current environment in UI (Under Assets tab)](" + model_usecase_ui_url + ")"))

2022/12/12 18:02:59 INFO : Asset successfully moved from operate to validate environment


In [21]:
time.sleep(3)
wml_model.set_environment_type(from_container="validate",to_container="test")

# uncomment if running cell by cell to see changes in UI

# model_usecase_ui_url = model_usecase.get_info(verbose=True)["url"]
# display(Markdown("[Click here to see the model current environment in UI (Under Assets tab)](" + model_usecase_ui_url + ")"))

2022/12/12 18:03:05 INFO : Asset successfully moved from validate to test environment


## Cleanup <a href="#clean"></a>

In [22]:
if run_cleanup_at_end:
    # remove model and model usecase
    model_usecase=wml_model.get_tracking_model_usecase()
    wml_model.remove_tracking_model_usecase()
    facts_client.assets.remove_asset(asset_id=model_usecase.get_info()["model_usecase_id"],container_type=model_usecase.get_info()["container_type"],container_id=model_usecase.get_info()["catalog_id"])
    facts_client.assets.remove_asset(asset_id=wml_model.get_info()["asset_id"],container_type=wml_model.get_info()["container_type"],container_id=wml_model.get_info()["container_id"])

else:
    
    model_ui_url = wml_model.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created wml model details in UI](" + model_ui_url + ")"))
    model_usecase_ui_url = model_usecase.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created model use case in UI](" + model_usecase_ui_url + ")"))

[Click here to see the created wml model details in UI](https://dataplatform.cloud.ibm.com/ml-runtime/models/72d97d15-1e9c-4246-95ac-fbfdb519168c?space_id=2f3d2c21-9b7e-4d8f-af5d-c65276c71318&context=cpdaas)

[Click here to see the created model use case in UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/aba647a6-f4ba-4133-bbe9-9e1c86f1263d?context=cpdaas)

[back to top](#introduction)

Congratulations. You have successfully finished the end to end sample around get/set environment pillars helpers in model usecase.

Copyright © 2020, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.